In [ ]:
import utils
reload(utils)
from utils import *

In [ ]:
from keras.datasets import imdb

In [ ]:
idx = imdb.get_word_index()

In [ ]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

In [ ]:
idx2word = { v: k for k, v in idx.iteritems() }

In [ ]:
path = "../data/imdb/imdb_full.pkl"
f = open(path, 'r')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [ ]:
' '.join([ idx2word[o] for o in x_train[0]])

In [ ]:
vocab_size = 5000

In [ ]:
trn = [ np.array([i if i < vocab_size - 1 else vocab_size - 1 for i in s]) for s in x_train ]
test = [ np.array([i if i < vocab_size - 1 else vocab_size - 1 for i in s]) for s in x_test ]

In [ ]:
lens = np.array(map(len, trn))
(lens.max(), lens.min(), lens.mean(), lens.std())
len(np.where(lens > 500)[0])

In [ ]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

In [ ]:
trn.shape

### Create simple model

In [ ]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

In [ ]:
model.summary()

### Single conv layer with max pooling

In [ ]:
?Dense

In [ ]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Convolution1D(64, 3, activation='relu'),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
conv1.summary()

In [ ]:
conv1.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=["accuracy"])

In [ ]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

### Pre-trained vectors

In [ ]:
dataset_path = '../data/glove/'

In [ ]:
def load_vectors(name):
    return (
        load_array(dataset_path + name + '.dat'),
        pickle.load(open(dataset_path + name + "_words.pkl")),
        pickle.load(open(dataset_path + name + "_idx.pkl")),
    )

In [ ]:
vecs, words, wordidx = load_vectors('6B.100d')

In [ ]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [ ]:
emb = create_emb()

In [ ]:
conv2 = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, weights=[emb], trainable=False),
    Convolution1D(64, 3, activation='relu'),
    ZeroPadding1D(1),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.6),
    Dense(1, activation='sigmoid')
])
conv1.summary()

In [ ]:
conv2.compile(loss="binary_crossentropy", optimizer=Adam(lr=0.0005), metrics=["accuracy"])

In [ ]:
conv2.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

In [ ]:
conv2.optimizer.lr = 0.00001

In [ ]:
conv3 = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [ ]:
conv3.compile(loss="binary_crossentropy", optimizer=Adam(lr=0.0005), metrics=["accuracy"])

In [ ]:
conv3.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

In [ ]:
conv3.optimizer.lr=1e-4

In [ ]:
conv3.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

### Multi-size CNN

In [ ]:
from keras.layers import Merge

In [ ]:
graph_in = Input ((vocab_size, 100))
convs = [ ] 
for fsz in range (3, 6): 
    x = Convolution1D(64, fsz, border_mode='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = Merge(mode="concat")(convs) 
graph = Model(graph_in, out) 

In [ ]:
emb = create_emb()

In [ ]:
model = Sequential ([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=1, batch_size=64)

In [ ]:
model.optimizer.lr = 1E-4

### LSTM

In [ ]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              W_regularizer=l2(1e-6), dropout=0.2),
    LSTM(100, consume_less='gpu'),
    Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)